# Homework DLT

## Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client

In [1]:
# !pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

What's the version of dlt that you installed?

In [2]:
import dlt
import requests
dlt.__version__

'1.13.0'

In [3]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

## Question 2. dlt pipeline

Now let's create a pipeline

How many rows were inserted into the zoomcamp_data collection?

In [4]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [5]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-09 15:54:42.134309+00:00 and COMPLETED in 15.17 seconds with 4 steps.
Step extract COMPLETED in 1.42 seconds.

Load package 1752076487.6334708 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)

Load package 1752076487.6334708 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 8.18 seconds.
Pipeline zoomcamp_pipeline load step completed in 8.16 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/timur/All/Projects/CoursesAndEducation/LLM Zoomcamp/LLM Zoomcamp HW/LLM-Zoomcamp/db.qdrant location to store data
Load package 1752076487.6334708 is LOADED and contains no failed jobs

Step run COMPLETED in 15.17 seconds.
Pipeline zoomcamp_pipeline load step completed in 8.16 seconds
1 load package(s) w

## Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

In [6]:
import json

with open('./db.qdrant/meta.json', 'r') as jf:
    meta_json = json.load(jf)

    for c in meta_json['collections'].items():
        print(list(c[1]['vectors'].keys())[0])

fast-bge-small-en
fast-bge-small-en
fast-bge-small-en
fast-bge-small-en
fast-bge-small-en
